# SBTi-Finance Tool - Portfolio Coverage Calculator

This notebook calculates portfolio coverage — the percentage of your investment portfolio that holds companies with validated Science Based Targets (SBTs).

**Supports both SBTi financial institution frameworks:**
- **FINT (FI Near-Term Targets)** — All validated SBT targets, temperature agnostic
- **FINZ (FI Net-Zero Standard)** — Only 1.5°C validated targets (per Implementation List)

**Coverage is measured two ways:**
1. **By Investment Value ($)** — What % of your portfolio's value is in SBT companies
2. **By Company Count (#)** — What % of companies in your portfolio have SBTs

This notebook does not calculate temperature scores or use weighted aggregation methods (WATS). For temperature scoring, use the temperature rating notebooks.

# Quick Start Guide

## What This Notebook Does
This notebook calculates **portfolio coverage** - the percentage of your investment portfolio that holds companies with validated Science Based Targets (SBTs).

This tool supports reporting for **two SBTi frameworks for financial institutions:**

| Framework | Abbreviation | What Counts | Use Case |
|-----------|--------------|-------------|----------|
| **FI Near-Term Targets** | FINT | All validated SBT targets (1.5°C, WB2°C, 2°C) | Near-term target setting and progress tracking |
| **FI Net-Zero Standard** | FINZ | Only 1.5°C validated targets | Net-zero commitment and Implementation List reporting |

---

## Results Categories

| Category | FINT | FINZ | Description |
|----------|------|------|-------------|
| **In Transition** | ✓ | ✓ | Companies with validated 1.5°C targets (per FINZ Implementation List Table 1) |
| **Other SBT** | ✓ | — | Companies with WB2°C or 2°C validated targets |
| **Assessed** | — | — | Companies without validated SBT targets |

**Coverage Metrics:**
- **FINT Coverage** = In Transition + Other SBT (all validated targets, temperature agnostic)
- **FINZ Coverage** = In Transition only (1.5°C validated targets per SBTi Target Status methodology)

---

## How to Use This Notebook

### Step 1: Run the Setup
Run the first few cells to install packages and load sample data.

### Step 2: Upload Your Data (optional)
To analyze your own portfolio:
1. Click the folder icon in the left sidebar
2. Navigate to `data/` folder
3. Upload your portfolio file (CSV or Excel)
4. Update the file path in the "Load Your Portfolio" section

### Step 3: Run All Cells
Click **Runtime > Run all** from the menu, or run cells one by one.

### Step 4: Download Results
Your results will be saved as an Excel file in the `data/` folder.

---

## Required Data Format
Your portfolio file needs these columns:

| Column | Required | Example |
|--------|----------|---------|
| `company_name` | Yes | "Apple Inc." |
| `company_id` | Yes | "AAPL" |
| `investment_value` | Yes | 1000000 |
| `isin` | Recommended | "US0378331005" |

---

In [ ]:
# =============================================================================
# SETUP - Run this cell first
# =============================================================================

# Install required package (only needed in Google Colab)
import sys
if 'google.colab' in sys.modules:
    print("Installing SBTi Finance Tool...")
    !pip install -q sbti-finance-tool
    print("Installation complete")
else:
    print("Running locally - using installed packages")

# Import required libraries
import pandas as pd
import openpyxl
from datetime import datetime
import os
import re
import warnings

# Suppress common warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=FutureWarning)

print("All packages loaded successfully")
print(f"Environment: {'Google Colab' if 'google.colab' in sys.modules else 'Local'}")

## Download Sample Data
This cell downloads example data so you can test the notebook. Skip this if using your own data.

In [ ]:
# Download sample portfolio data
import urllib.request

if not os.path.isdir("data"):
    os.mkdir("data")
    
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve(
        "https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/example_portfolio.csv", 
        "data/example_portfolio.csv"
    )
    print("Sample data downloaded to data/example_portfolio.csv")
else:
    print("Sample data already exists")

## USER INPUT: Load Your Portfolio

**To use your own data:**
1. Upload your file to the `data/` folder (click folder icon in left sidebar)
2. Change the filename below to match your file

**To use sample data:** Just run this cell as-is.

In [ ]:
df_portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")
#df_portfolio = pd.read_excel("data/example_portfolio.xlsx", engine="openpyxl") # .xlsx format

#Use your local file instead
#my_file_path = "path/to/your/portfolio_file.csv"
#df_portfolio = pd.read_csv(my_file_path, encoding="utf-8")

In [ ]:
# Standardize column names (handles different naming formats automatically)
def convert_to_snake_case(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    s3 = s2.lower()
    s4 = re.sub(r'[^a-z0-9_]', '_', s3)
    s5 = re.sub(r'_+', '_', s4)
    return s5.strip('_')

df_portfolio.columns = [convert_to_snake_case(col) for col in df_portfolio.columns]
print(f"Loaded {len(df_portfolio)} companies from portfolio")

In [ ]:
# Auto-detect column formats (ISIN, LEI identifiers)
portfolio_isin_col = next((c for c in ['company_isin', 'isin'] if c in df_portfolio.columns), None)
portfolio_lei_col = next((c for c in ['company_lei', 'lei'] if c in df_portfolio.columns), None)

# Show what identifiers were found
id_status = []
if portfolio_isin_col:
    id_status.append("ISIN")
if portfolio_lei_col:
    id_status.append("LEI")
if id_status:
    print(f"Company identifiers found: {', '.join(id_status)}")
else:
    print("Warning: No ISIN or LEI columns found - will match by company name only")

In [ ]:
# Validate portfolio data
required_cols = ['company_name', 'company_id', 'investment_value']
missing_cols = [col for col in required_cols if col not in df_portfolio.columns]

if missing_cols:
    print(f"ERROR: Missing required columns: {missing_cols}")
    print(f"Your file needs: company_name, company_id, investment_value")
    raise ValueError(f"Missing columns: {missing_cols}")
else:
    total_value = df_portfolio['investment_value'].sum()
    print(f"Portfolio validated: {len(df_portfolio)} companies, ${total_value:,.0f} total value")

## USER INPUT: Select Analysis Date

Choose the date for your analysis. This determines which SBTi targets are included (only targets published by this date).

**Default:** December 31, 2025

In [ ]:
year = 2025 #enter the year for which you want to calculate the portfolio coverage
month = 12 #enter the month for which you want to calculate the portfolio coverage
day = 31 #enter the day for which you want to calculate the portfolio coverage

In [ ]:
user_date = datetime(year, month, day)

## Loading SBTi Database
The tool automatically downloads the latest list of companies with validated Science Based Targets.

In [ ]:
# Load SBTi Companies Taking Action database
from SBTi.data.sbti import SBTi

print("Loading SBTi database (this may take a moment)...")
sbti_provider = SBTi()
cta_file = sbti_provider.targets.copy()

# Count unique companies with targets
companies_with_targets_count = len(cta_file[cta_file[sbti_provider.c.COL_ACTION] == sbti_provider.c.VALUE_ACTION_TARGET][sbti_provider.c.COL_COMPANY_NAME].unique())
print(f"Loaded SBTi database: {companies_with_targets_count:,} companies with validated targets")

## Processing Data
Filtering SBTi data to match your analysis date...

In [ ]:
# Filter for companies with validated targets
targets = cta_file.copy()
companies_with_targets = targets[targets[sbti_provider.c.COL_ACTION] == sbti_provider.c.VALUE_ACTION_TARGET]

print(f"Processing {len(companies_with_targets[sbti_provider.c.COL_COMPANY_NAME].unique()):,} companies with SBT targets")

In [ ]:
# Handle date column format variations (contingency for different CTA formats)
date_col = sbti_provider.c.COL_DATE_PUBLISHED
potential_date_cols = ['date_updated', 'Date Updated', 'date_published', 'Date Published']

# Check if expected date column exists, otherwise search for alternatives
if date_col not in companies_with_targets.columns:
    found_date_col = next((col for col in potential_date_cols if col in companies_with_targets.columns), None)
    if found_date_col:
        companies_with_targets = companies_with_targets.rename(columns={found_date_col: date_col})
        print(f"Date column mapped: '{found_date_col}' -> '{date_col}'")
    else:
        print(f"WARNING: No date column found. Available columns: {list(companies_with_targets.columns)}")
        print("Date filtering will be skipped.")
else:
    print(f"Date column: '{date_col}'")

In [ ]:
# Apply date filter
df_targets = companies_with_targets.copy()
df_targets[sbti_provider.c.COL_DATE_PUBLISHED] = pd.to_datetime(df_targets[sbti_provider.c.COL_DATE_PUBLISHED])

# Filter by user-specified date
date_filtered_df = df_targets.loc[df_targets[sbti_provider.c.COL_DATE_PUBLISHED] <= user_date]

# Name normalization function (handles whitespace variations)
def normalize_name(name):
    """Normalize company name for matching: lowercase, collapse whitespace."""
    if pd.isna(name):
        return None
    return ' '.join(str(name).lower().split())

# Create normalized company name set from ALL date-filtered companies
date_filtered_df['company_name_normalized'] = date_filtered_df[sbti_provider.c.COL_COMPANY_NAME].apply(normalize_name)
company_name_set = set(date_filtered_df['company_name_normalized'].dropna())

# Create ISIN/LEI sets only from companies that have these identifiers
isin_set = set(date_filtered_df[sbti_provider.c.COL_COMPANY_ISIN].dropna())
lei_set = set(date_filtered_df[sbti_provider.c.COL_COMPANY_LEI].dropna())

# Keep filtered_df for 1.5C analysis
filtered_df = date_filtered_df.copy()

print(f"Filtered to targets published by {user_date.strftime('%B %d, %Y')}")
print(f"  - Companies for name matching: {len(company_name_set):,}")
print(f"  - Unique ISINs: {len(isin_set):,}")
print(f"  - Unique LEIs: {len(lei_set):,}")

## Matching Your Portfolio
Now matching your portfolio companies against the SBTi database...

In [ ]:
# =============================================================================
# COMPANY MATCHING UTILITIES
# =============================================================================

def match_company(row, target_isin_set, target_lei_set, target_name_set):
    """
    Match a portfolio company against a set of target companies.
    
    Returns the match method ('LEI', 'ISIN', 'Name') or None if no match.
    
    Priority order (highest reliability first):
    1. LEI match (Legal Entity Identifier - most reliable)
    2. ISIN match (International Securities ID - very reliable)
    3. Company name match (normalized, case-insensitive - fallback)
    """
    # Check LEI first (most reliable identifier)
    if portfolio_lei_col and pd.notna(row.get(portfolio_lei_col)):
        if row.get(portfolio_lei_col) in target_lei_set:
            return 'LEI'

    # Check ISIN second (very reliable)
    if portfolio_isin_col and pd.notna(row.get(portfolio_isin_col)):
        if row.get(portfolio_isin_col) in target_isin_set:
            return 'ISIN'

    # Check company name last (normalized match)
    if pd.notna(row.get('company_name')):
        # Use same normalization as CTA names
        normalized_name = normalize_name(row.get('company_name'))
        if normalized_name and normalized_name in target_name_set:
            return 'Name'

    return None

# =============================================================================
# VALIDATE PORTFOLIO AGAINST ALL SBT TARGETS (DATE-FILTERED) - FINT
# =============================================================================

# Apply matching to determine if company has validated SBT target
df_portfolio['match_method_all'] = df_portfolio.apply(
    lambda row: match_company(row, isin_set, lei_set, company_name_set), 
    axis=1
)
df_portfolio['validated'] = df_portfolio['match_method_all'].notna()

# Diagnostic: Show match method breakdown
match_counts = df_portfolio['match_method_all'].value_counts()
total_matched = df_portfolio['validated'].sum()
total_companies = len(df_portfolio)

print(f"SBT Matching (FINT): {total_matched} / {total_companies} companies matched")
print(f"  Match breakdown:")
for method in ['LEI', 'ISIN', 'Name']:
    count = match_counts.get(method, 0)
    if count > 0:
        print(f"    - {method}: {count}")

## USER INPUT: 1.5°C Target Settings (Optional)

**What is a 1.5°C target?** Companies with the most ambitious climate commitments, aligned with limiting global warming to 1.5°C. These are the only targets that count toward **FINZ (FI Net-Zero Standard)** coverage.

**Default settings work for most users.** Only change if you need specific filtering.

In [ ]:
# 1.5°C Target Configuration (most users can leave defaults)
# --------------------------------------------------------
# INCLUDE_PURE_15C = True   -> Count companies with pure 1.5°C targets
# INCLUDE_MIXED_15C_2C = False -> Exclude mixed targets like "1.5°C/2°C"

INCLUDE_PURE_15C = True        # Recommended: True
INCLUDE_MIXED_15C_2C = False   # Recommended: False (more conservative)

print(f"1.5°C settings: Pure={INCLUDE_PURE_15C}, Mixed={INCLUDE_MIXED_15C_2C}")

In [ ]:
# Identify 1.5°C aligned companies in SBTi database (for FINZ coverage)
df_analysis = filtered_df.copy()

# Find Target Classification column (handle potential naming variations)
TARGET_CLASSIFICATION_COL = None
potential_tc_cols = ['Target Classification', 'target_classification', 'Near Term Classification', 
                     'near_term_target_classification', 'target_classification_short']
for col in potential_tc_cols:
    if col in df_analysis.columns:
        TARGET_CLASSIFICATION_COL = col
        break

if TARGET_CLASSIFICATION_COL is None:
    print("ERROR: No Target Classification column found!")
    print(f"Available columns: {list(df_analysis.columns)}")
    raise ValueError("Missing Target Classification column")

# Show available classification values for transparency
unique_classifications = df_analysis[TARGET_CLASSIFICATION_COL].dropna().unique()
print(f"Target Classification column: '{TARGET_CLASSIFICATION_COL}'")
print(f"Classifications found: {sorted([str(x) for x in unique_classifications])}")

# Define known 1.5°C classification patterns (per FINZ Implementation List Table 1)
PURE_15C_VALUES = ['1.5°C', '1.5°C/1.5°C']  # Pure 1.5°C (both near-term and long-term)
MIXED_15C_PATTERNS = ['1.5°C/Well-below 2°C', '1.5°C/2°C', '1.5/']  # Mixed with other temps

# Filter for 1.5°C targets based on settings
df_1_5c = pd.DataFrame()

if INCLUDE_PURE_15C:
    # Match known pure 1.5°C values
    pure_mask = df_analysis[TARGET_CLASSIFICATION_COL].isin(PURE_15C_VALUES)
    pure_15c = df_analysis[pure_mask]
    df_1_5c = pd.concat([df_1_5c, pure_15c])
    print(f"  Pure 1.5°C matches: {len(pure_15c)} rows")
    
    # Check for unexpected pure 1.5°C patterns (safety check)
    unexpected_pure = df_analysis[
        (df_analysis[TARGET_CLASSIFICATION_COL].astype(str).str.startswith('1.5°C')) &
        (~df_analysis[TARGET_CLASSIFICATION_COL].isin(PURE_15C_VALUES)) &
        (~df_analysis[TARGET_CLASSIFICATION_COL].astype(str).str.contains('/', na=False))
    ]
    if len(unexpected_pure) > 0:
        print(f"  WARNING: Found {len(unexpected_pure)} rows with unexpected 1.5°C pattern")
        print(f"           Values: {unexpected_pure[TARGET_CLASSIFICATION_COL].unique()}")
    
if INCLUDE_MIXED_15C_2C:
    # Match mixed classifications containing 1.5°C with other temps
    mixed_mask = (
        (df_analysis[TARGET_CLASSIFICATION_COL].astype(str).str.contains('1.5', na=False)) &
        (df_analysis[TARGET_CLASSIFICATION_COL].astype(str).str.contains('/', na=False)) &
        (~df_analysis[TARGET_CLASSIFICATION_COL].isin(PURE_15C_VALUES))
    )
    mixed = df_analysis[mixed_mask]
    df_1_5c = pd.concat([df_1_5c, mixed])
    print(f"  Mixed 1.5°C matches: {len(mixed)} rows")

# Create lookup sets for 1.5°C matching
if len(df_1_5c) > 0:
    # Deduplicate by company name to count unique companies
    df_1_5c_unique = df_1_5c.drop_duplicates(subset=[sbti_provider.c.COL_COMPANY_NAME])
    
    # Create ISIN/LEI sets from companies that have these identifiers
    isin_set_1_5c = set(df_1_5c[sbti_provider.c.COL_COMPANY_ISIN].dropna())
    lei_set_1_5c = set(df_1_5c[sbti_provider.c.COL_COMPANY_LEI].dropna())
    
    # Create normalized name set from ALL 1.5°C companies
    company_name_set_1_5c = set(df_1_5c[sbti_provider.c.COL_COMPANY_NAME].apply(normalize_name).dropna())
    
    print(f"Found {len(df_1_5c_unique):,} unique companies with 1.5°C targets (FINZ eligible)")
    print(f"  - For name matching: {len(company_name_set_1_5c):,}")
    print(f"  - With ISIN: {len(isin_set_1_5c):,}")
    print(f"  - With LEI: {len(lei_set_1_5c):,}")
else:
    isin_set_1_5c, lei_set_1_5c, company_name_set_1_5c = set(), set(), set()
    print("Warning: No 1.5°C companies found with current settings")

In [ ]:
# =============================================================================
# MATCH PORTFOLIO COMPANIES TO 1.5°C ALIGNED COMPANIES (FINZ)
# =============================================================================

# Apply matching using the reusable match_company function
df_portfolio['match_method_1_5c'] = df_portfolio.apply(
    lambda row: match_company(row, isin_set_1_5c, lei_set_1_5c, company_name_set_1_5c), 
    axis=1
)
df_portfolio['is_1_5c'] = df_portfolio['match_method_1_5c'].notna()

# Diagnostic: Show 1.5°C match method breakdown
match_counts_15c = df_portfolio['match_method_1_5c'].value_counts()
total_matched_15c = df_portfolio['is_1_5c'].sum()
total_portfolio_companies = len(df_portfolio)

print(f"1.5°C Matching (FINZ): {total_matched_15c} / {total_portfolio_companies} companies")
if total_matched_15c > 0:
    print(f"  Match breakdown:")
    for method in ['LEI', 'ISIN', 'Name']:
        count = match_counts_15c.get(method, 0)
        if count > 0:
            print(f"    - {method}: {count}")

---
# Your Results

The following sections show your portfolio's climate alignment for **both SBTi financial institution frameworks:**

| Framework | Abbreviation | Coverage Includes |
|-----------|--------------|-------------------|
| **FI Near-Term Targets** | FINT | In Transition + Other SBT (all validated targets, temp agnostic) |
| **FI Net-Zero Standard** | FINZ | In Transition only (1.5°C targets per Implementation List Table 1) |

**Category Key:**
- **In Transition** — 1.5°C validated targets (counts for FINT and FINZ)
- **Other SBT** — WB2°C, 2°C validated targets (counts for FINT only)
- **Assessed** — No validated SBT target (does not count for either framework)

In [ ]:
# =============================================================================
# CALCULATE ALL COVERAGE METRICS
# =============================================================================

total_investment_value = df_portfolio['investment_value'].sum()
total_companies = len(df_portfolio)
distinct_company_count = df_portfolio['company_name'].nunique()

# All SBT coverage (FINT - temp agnostic)
sbt_investment_value = df_portfolio.loc[df_portfolio['validated'] == True, 'investment_value'].sum()
sbt_company_count = df_portfolio['validated'].sum()

# 1.5°C coverage (FINZ - 1.5°C specificity)
val_1_5c = df_portfolio.loc[df_portfolio['is_1_5c'] == True, 'investment_value'].sum()
count_1_5c = df_portfolio['is_1_5c'].sum()

# Calculate percentages
coverage_fint = (sbt_investment_value / total_investment_value * 100) if total_investment_value > 0 else 0
coverage_finz = (val_1_5c / total_investment_value * 100) if total_investment_value > 0 else 0

# Display results
print("=" * 70)
print("                    PORTFOLIO COVERAGE SUMMARY")
print("=" * 70)
print(f"\nTotal Portfolio Value: ${total_investment_value:,.0f}")
print(f"Total Companies: {distinct_company_count}")

print(f"\n" + "=" * 70)
print("  FRAMEWORK COVERAGE METRICS")
print("=" * 70)

print(f"\n  FINT - FI NEAR-TERM TARGETS (all validated SBT, temp agnostic)")
print(f"  ----------------------------------------------------------------")
print(f"    Coverage:  {coverage_fint:.1f}% of portfolio value")
print(f"    Value:     ${sbt_investment_value:,.0f}")
print(f"    Companies: {sbt_company_count}")

print(f"\n  FINZ - FI NET-ZERO STANDARD (1.5°C validated targets only)")
print(f"  ----------------------------------------------------------------")
print(f"    Coverage:  {coverage_finz:.1f}% of portfolio value")
print(f"    Value:     ${val_1_5c:,.0f}")
print(f"    Companies: {count_1_5c}")

print(f"\n" + "=" * 70)
print("  BREAKDOWN BY CATEGORY")
print("=" * 70)
print(f"\n  IN TRANSITION (1.5°C targets) — counts for FINT and FINZ")
print(f"    Value:     ${val_1_5c:,.0f} ({coverage_finz:.1f}%)")
print(f"    Companies: {count_1_5c}")
print(f"\n  OTHER SBT (WB2°C, 2°C targets) — counts for FINT only")
print(f"    Value:     ${sbt_investment_value - val_1_5c:,.0f} ({coverage_fint - coverage_finz:.1f}%)")
print(f"    Companies: {sbt_company_count - count_1_5c}")
print(f"\n  ASSESSED (no SBT target) — does not count for either framework")
print(f"    Value:     ${total_investment_value - sbt_investment_value:,.0f} ({100 - coverage_fint:.1f}%)")
print(f"    Companies: {total_companies - sbt_company_count}")
print(f"\n" + "=" * 70)

In [ ]:
# =============================================================================
# VISUAL SUMMARY - Climate Alignment Chart
# =============================================================================

import matplotlib.pyplot as plt

# Classify each company per SBTi frameworks
def get_climate_alignment(row):
    """
    Classify company climate alignment for FINT/FINZ frameworks.
    
    - In Transition: 1.5°C validated targets (counts for FINT and FINZ)
    - Other SBT: WB2°C, 2°C targets (counts for FINT only)
    - Assessed: No validated SBT target
    """
    if row['is_1_5c']:
        return "In Transition"
    elif row['validated']:
        return "Other SBT"
    else:
        return "Assessed"

df_portfolio['climate_alignment'] = df_portfolio.apply(get_climate_alignment, axis=1)

# Calculate values for chart
alignment_values = df_portfolio.groupby('climate_alignment')['investment_value'].sum()

# Ensure order and handle missing categories
categories = ['In Transition', 'Other SBT', 'Assessed']
values = [alignment_values.get(cat, 0) for cat in categories]
colors = ['#2ecc71', '#f39c12', '#95a5a6']  # Green, Orange, Gray

# Create charts
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Pie chart by value
wedges, texts, autotexts = ax1.pie(
    values, 
    labels=categories, 
    autopct=lambda pct: f'{pct:.1f}%' if pct > 0 else '',
    colors=colors,
    startangle=90
)
ax1.set_title('Portfolio by Investment Value', fontsize=14, fontweight='bold')

# Bar chart by company count
counts = df_portfolio.groupby('climate_alignment')['company_name'].nunique()
count_values = [counts.get(cat, 0) for cat in categories]

bars = ax2.bar(categories, count_values, color=colors)
ax2.set_ylabel('Number of Companies')
ax2.set_title('Portfolio by Company Count', fontsize=14, fontweight='bold')

# Add value labels on bars
for bar, count in zip(bars, count_values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             str(count), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Print summary table with framework applicability
print("\n" + "=" * 75)
print("              CLIMATE ALIGNMENT BY CATEGORY")
print("=" * 75)
print(f"\n{'Category':<18} {'FINT':>10} {'FINZ':>10} {'Value':>15} {'%':>8} {'Co.':>6}")
print("-" * 75)
framework_fint = ['Yes', 'Yes', 'No']
framework_finz = ['Yes', 'No', 'No']
for cat, fint, finz, val, cnt in zip(categories, framework_fint, framework_finz, values, count_values):
    pct = (val / total_investment_value * 100) if total_investment_value > 0 else 0
    print(f"{cat:<18} {fint:>10} {finz:>10} ${val:>13,.0f} {pct:>7.1f}% {cnt:>6}")
print("-" * 75)
print(f"{'TOTAL':<18} {'':<10} {'':<10} ${total_investment_value:>13,.0f} {'100.0':>7}% {distinct_company_count:>6}")

# Framework totals
print("\n" + "-" * 75)
print("FRAMEWORK COVERAGE TOTALS:")
print(f"  FINT (FI Near-Term): {coverage_fint:.1f}% (In Transition + Other SBT)")
print(f"  FINZ (FI Net-Zero):  {coverage_finz:.1f}% (In Transition only)")

---
# Download Your Results

Run the cell below to save your results as an Excel file.

**Output location:** `data/portfolio_climate_alignment.xlsx`

| Environment | How to Access Your File |
|-------------|------------------------|
| **Google Colab** | Click folder icon in left sidebar > Navigate to `data/` > Right-click file > Download |
| **Local/Jupyter** | File is saved to the `data/` folder in your notebook directory |

**What's included:**
- **Portfolio sheet:** Your companies with climate alignment classification
- **Summary sheet:** Coverage metrics for both FINT (Near-Term) and FINZ (Net-Zero) frameworks

In [ ]:
# =============================================================================
# SAVE RESULTS TO EXCEL
# =============================================================================

output_filename = 'portfolio_climate_alignment'
output_dir = 'data'

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

# Prepare clean export data with match method for verification
export_columns = ['company_id', 'company_name', 'investment_value', 'climate_alignment', 'match_method_all']
if portfolio_isin_col:
    export_columns.insert(3, portfolio_isin_col)
if portfolio_lei_col:
    export_columns.insert(3, portfolio_lei_col)

export_columns = [c for c in export_columns if c in df_portfolio.columns]
df_export = df_portfolio[export_columns].copy()

# Rename match_method_all to clearer name
if 'match_method_all' in df_export.columns:
    df_export = df_export.rename(columns={'match_method_all': 'match_method'})

# Calculate category values for summary
in_transition_value = df_portfolio.loc[df_portfolio['climate_alignment'] == 'In Transition', 'investment_value'].sum()
in_transition_count = (df_portfolio['climate_alignment'] == 'In Transition').sum()
other_sbt_value = df_portfolio.loc[df_portfolio['climate_alignment'] == 'Other SBT', 'investment_value'].sum()
other_sbt_count = (df_portfolio['climate_alignment'] == 'Other SBT').sum()
assessed_value = df_portfolio.loc[df_portfolio['climate_alignment'] == 'Assessed', 'investment_value'].sum()
assessed_count = (df_portfolio['climate_alignment'] == 'Assessed').sum()

in_transition_pct = (in_transition_value / total_investment_value * 100) if total_investment_value > 0 else 0
other_sbt_pct = (other_sbt_value / total_investment_value * 100) if total_investment_value > 0 else 0
assessed_pct = (assessed_value / total_investment_value * 100) if total_investment_value > 0 else 0

# Create summary data with both FINT/FINZ framework metrics
summary_data = {
    'Metric': [
        'Analysis Date', 'Coverage Date', '',
        'Total Portfolio Value', 'Total Companies', '',
        '--- FRAMEWORK COVERAGE ---', '', '',
        'FINT (FI NEAR-TERM TARGETS)', '  Description', '  Coverage %', '  Value', '  Companies', '',
        'FINZ (FI NET-ZERO STANDARD)', '  Description', '  Coverage %', '  Value', '  Companies', '',
        '--- CATEGORY BREAKDOWN ---', '', '',
        'In Transition (1.5°C targets)', '  Counts for FINT', '  Counts for FINZ', '  Value', '  Percentage', '  Companies', '',
        'Other SBT (WB2°C, 2°C targets)', '  Counts for FINT', '  Counts for FINZ', '  Value', '  Percentage', '  Companies', '',
        'Assessed (no SBT target)', '  Counts for FINT', '  Counts for FINZ', '  Value', '  Percentage', '  Companies'
    ],
    'Value': [
        datetime.now().strftime('%Y-%m-%d'), user_date.strftime('%Y-%m-%d'), '',
        f"${total_investment_value:,.0f}", str(distinct_company_count), '',
        '', '', '',
        'All validated SBT targets (temp agnostic)', '', f"{coverage_fint:.1f}%", f"${sbt_investment_value:,.0f}", str(sbt_company_count), '',
        '1.5°C validated targets only', '', f"{coverage_finz:.1f}%", f"${val_1_5c:,.0f}", str(count_1_5c), '',
        '', '', '',
        '', 'Yes', 'Yes', f"${in_transition_value:,.0f}", f"{in_transition_pct:.1f}%", str(in_transition_count), '',
        '', 'Yes', 'No', f"${other_sbt_value:,.0f}", f"{other_sbt_pct:.1f}%", str(other_sbt_count), '',
        '', 'No', 'No', f"${assessed_value:,.0f}", f"{assessed_pct:.1f}%", str(assessed_count)
    ]
}

# Save to Excel
excel_path = f"{output_dir}/{output_filename}.xlsx"

with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    df_export.to_excel(writer, sheet_name='Portfolio', index=False)
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

# Get absolute path for local users
abs_path = os.path.abspath(excel_path)

print("=" * 60)
print("         RESULTS SAVED SUCCESSFULLY")
print("=" * 60)
print(f"\nFile saved to: {excel_path}")

if 'google.colab' in sys.modules:
    print(f"\n[Google Colab] To download your file:")
    print(f"   1. Click the folder icon in the left sidebar")
    print(f"   2. Navigate to 'data' folder")
    print(f"   3. Right-click '{output_filename}.xlsx' > Download")
else:
    print(f"\n[Local] Full path: {abs_path}")
    print(f"   Open this location in your file explorer to access the file.")

print("\n" + "=" * 60)

# Show preview with match method for verification
print(f"\nPreview of exported data (with match_method for verification):")
print(df_export.head(10).to_string(index=False))